In [1]:
import numpy as np
import pandas as pd
import rdkit

In [2]:
import pandas as pd
sol = pd.read_csv('solubility.csv')
sol

,measured log(solubility:mol/L),SMILES
0,-2.180,ClCC(Cl)(Cl)Cl
1,-2.000,CC(Cl)(Cl)Cl
2,-1.740,ClC(Cl)C(Cl)Cl
3,-1.480,ClCC(Cl)Cl
4,-3.040,FC(F)(Cl)C(F)(Cl)Cl
...,...,...
1139,1.144,CNC(=O)C(C)SCCSP(=O)(OC)(OC)
1140,-4.925,CC1(OC(=O)N(C1=O)c2cc(Cl)cc(Cl)c2)C=C
1141,-3.893,CC(=O)CC(c1ccccc1)c3c(O)c2ccccc2oc3=O
1142,-3.790,Cc1cccc(C)c1NC(=O)c2cc(c(Cl)cc2O)S(N)(=O)=O


In [3]:
sol.columns

Index(['measured log(solubility:mol/L)', 'SMILES'], dtype='object')

In [4]:
from rdkit import Chem
m = Chem.MolFromSmiles(sol.SMILES[0])
m.GetNumAtoms()

6

In [5]:
mol_list2 = [Chem.MolFromSmiles(element) for element in sol.SMILES]
len(mol_list2)

1144

In [6]:
import numpy as np
from rdkit.Chem import Descriptors

def generate(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)
        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MolLogP","MolWt","NumRotatableBonds"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [7]:
df = generate(sol.SMILES)
df

,MolLogP,MolWt,NumRotatableBonds
0,2.59540,167.850,0.0
1,2.37650,133.405,0.0
2,2.59380,167.850,1.0
3,2.02890,133.405,1.0
4,2.91890,187.375,1.0
...,...,...,...
1139,1.98820,287.343,8.0
1140,3.42130,286.114,2.0
1141,3.60960,308.333,4.0
1142,2.56214,354.815,3.0


In [8]:
def AromaticAtoms(m):
    aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
    aa_count = []
    for i in aromatic_atoms:
        if i==True:
            aa_count.append(1)
    sum_aa_count = sum(aa_count)
    return sum_aa_count

In [9]:
desc_AromaticAtoms = [AromaticAtoms(element) for element in mol_list2]
len(desc_AromaticAtoms)

1144

In [10]:
desc_HeavyAtomCount = [Descriptors.HeavyAtomCount(element) for element in mol_list2]
len(desc_HeavyAtomCount)

1144

In [11]:
desc_AromaticProportion = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list2]
len(desc_AromaticProportion)

1144

In [12]:
df_desc_AromaticProportion = pd.DataFrame(desc_AromaticProportion, columns=['AromaticProportion'])
df_desc_AromaticProportion

,AromaticProportion
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
1139,0.000000
1140,0.333333
1141,0.695652
1142,0.521739


In [13]:
df

,MolLogP,MolWt,NumRotatableBonds
0,2.59540,167.850,0.0
1,2.37650,133.405,0.0
2,2.59380,167.850,1.0
3,2.02890,133.405,1.0
4,2.91890,187.375,1.0
...,...,...,...
1139,1.98820,287.343,8.0
1140,3.42130,286.114,2.0
1141,3.60960,308.333,4.0
1142,2.56214,354.815,3.0


In [14]:
X = pd.concat([df,df_desc_AromaticProportion], axis=1)
X

,MolLogP,MolWt,NumRotatableBonds,AromaticProportion
0,2.59540,167.850,0.0,0.000000
1,2.37650,133.405,0.0,0.000000
2,2.59380,167.850,1.0,0.000000
3,2.02890,133.405,1.0,0.000000
4,2.91890,187.375,1.0,0.000000
...,...,...,...,...
1139,1.98820,287.343,8.0,0.000000
1140,3.42130,286.114,2.0,0.333333
1141,3.60960,308.333,4.0,0.695652
1142,2.56214,354.815,3.0,0.521739


In [19]:
Y = sol['measured log(solubility:mol/L)']
Y

0      -2.180
1      -2.000
2      -1.740
3      -1.480
4      -3.040
        ...  
1139    1.144
1140   -4.925
1141   -3.893
1142   -3.790
1143   -2.581
Name: measured log(solubility:mol/L), Length: 1144, dtype: float64

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# Modelling

In [21]:
# Linear regressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
linear_reg = linear_model.LinearRegression()
linear_reg.fit(X_train, Y_train) 

LinearRegression()

In [22]:
linear_reg_pred = linear_reg.predict(X_test)
print('Mean squared error (MSE): %.2f'% mean_squared_error(Y_test, linear_reg_pred))
print('Coefficient of determination (R^2): %.2f'% r2_score(Y_test, linear_reg_pred))

Mean squared error (MSE): 0.90
Coefficient of determination (R^2): 0.79


In [23]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

In [24]:
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, Y_train) 
rf_reg_pred = rf_reg.predict(X_test)
print('Mean squared error (MSE): %.2f'% mean_squared_error(Y_test, rf_reg_pred))
print('Coefficient of determination (R^2): %.2f'% r2_score(Y_test, rf_reg_pred))

Mean squared error (MSE): 0.53
Coefficient of determination (R^2): 0.87


In [29]:
xg_reg = XGBRegressor(eval_metric = 'rmse',
    learning_rate =0.01,
 n_estimators=1000,
 max_depth=10,
 min_child_weight=1,
 gamma=0.2,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:squarederror',
 nthread=4,
 scale_pos_weight=1,
 eta = 0.1,                
 seed=27)

xg_reg.fit(X_train, Y_train) 
xg_reg_pred = xg_reg.predict(X_test)
print('Mean squared error (MSE): %.2f'% mean_squared_error(Y_test, xg_reg_pred))
print('Coefficient of determination (R^2): %.2f'% r2_score(Y_test, xg_reg_pred))

Mean squared error (MSE): 0.51
Coefficient of determination (R^2): 0.88


In [27]:
mlp_reg = MLPRegressor(validation_fraction=0.25, learning_rate='invscaling',max_iter=400,random_state = 50)
mlp_reg.fit(X_train, Y_train) 
mlp_reg_pred = mlp_reg.predict(X_test)
print('Mean squared error (MSE): %.2f'% mean_squared_error(Y_test,mlp_reg_pred))
print('Coefficient of determination (R^2): %.2f'% r2_score(Y_test, mlp_reg_pred))

Mean squared error (MSE): 0.69
Coefficient of determination (R^2): 0.84


## Tuning

In [57]:
from sklearn.model_selection import RandomizedSearchCV

import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 1500, stop = 1700, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(500, 600,20)]
# Minimum number of samples required to split a node
#min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
#min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               #'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf
}
print(random_grid)

{'n_estimators': [1500, 1510, 1521, 1531, 1542, 1552, 1563, 1573, 1584, 1594, 1605, 1615, 1626, 1636, 1647, 1657, 1668, 1678, 1689, 1700], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [500, 505, 510, 515, 521, 526, 531, 536, 542, 547, 552, 557, 563, 568, 573, 578, 584, 589, 594, 600]}


In [58]:
rf=RandomForestRegressor()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=5,verbose=2,
                               random_state=100,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train,Y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'max_depth': [500, 505, 510, 515, 521,
                                                      526, 531, 536, 542, 547,
                                                      552, 557, 563, 568, 573,
                                                      578, 584, 589, 594, 600],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'n_estimators': [1500, 1510, 1521, 1531,
                                                         1542, 1552, 1563, 1573,
                                                         1584, 1594, 1605, 1615,
                                                         1626, 1636, 1647, 1657,
                                                         1668, 1678, 1689,
                                           

In [59]:
rf_randomcv.best_params_


{'max_depth': 531, 'max_features': 'log2', 'n_estimators': 1668}

In [60]:
best_random_grid=rf_randomcv.best_estimator_
tuned_y_pred = best_random_grid.predict(X_test)
r2_score(Y_test,tuned_y_pred)

0.8736835177904761

In [22]:
from sklearn.model_selection import GridSearchCV

max_depth = [int(x) for x in np.linspace(start = 500, stop = 560, num = 10)]
max_features = ['log2','sqrt']
n_estimators = [int(x) for x in np.linspace(start = 1600, stop = 1700, num = 25)]

# Create the random grid
grid = {'max_depth': max_depth,
        'max_features': max_features,
        'n_estimators': n_estimators
}
print(grid)

{'max_depth': [500, 506, 513, 520, 526, 533, 540, 546, 553, 560], 'max_features': ['log2', 'sqrt'], 'n_estimators': [1600, 1604, 1608, 1612, 1616, 1620, 1625, 1629, 1633, 1637, 1641, 1645, 1650, 1654, 1658, 1662, 1666, 1670, 1675, 1679, 1683, 1687, 1691, 1695, 1700]}


In [23]:
rf = RandomForestRegressor()
grid_search=GridSearchCV(estimator=rf,param_grid=grid,cv=5,n_jobs=-1,verbose=3)
grid_search.fit(X_train,Y_train)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [500, 506, 513, 520, 526, 533, 540, 546,
                                       553, 560],
                         'max_features': ['log2', 'sqrt'],
                         'n_estimators': [1600, 1604, 1608, 1612, 1616, 1620,
                                          1625, 1629, 1633, 1637, 1641, 1645,
                                          1650, 1654, 1658, 1662, 1666, 1670,
                                          1675, 1679, 1683, 1687, 1691, 1695,
                                          1700]},
             verbose=3)

In [24]:
grid_search.best_params_

{'max_depth': 506, 'max_features': 'sqrt', 'n_estimators': 1620}

In [26]:
best_grid=grid_search.best_estimator_
tuned_y_pred1 = best_grid.predict(X_test)
r2_score(Y_test,tuned_y_pred1)

0.874633362489119

In [114]:
tuned_rf_reg = RandomForestRegressor(max_depth=560,max_features='sqrt',n_estimators = 100,min_samples_leaf=1,min_samples_split=3,n_jobs=-1)
tuned_rf_reg.fit(X_train, Y_train) 
tuned_rf_reg_pred = tuned_rf_reg.predict(X_test)
print('Mean squared error (MSE): %.2f'% mean_squared_error(Y_test, tuned_rf_reg_pred))
print('Coefficient of determination (R^2): %.2f'% r2_score(Y_test, tuned_rf_reg_pred))

Mean squared error (MSE): 0.55
Coefficient of determination (R^2): 0.87


In [115]:
print('Mean squared error (MSE): %.2f'% mean_squared_error(Y_train, tuned_rf_reg.predict(X_train),squared=False))
print('Coefficient of determination (R^2): %.2f'% r2_score(Y_train, tuned_rf_reg.predict(X_train)))

Mean squared error (MSE): 0.32
Coefficient of determination (R^2): 0.98


In [28]:
import pickle as pk
# save the model to disk
filename = 'solubility_finalized_model.sav'
pk.dump(xg_reg, open(filename, 'wb'))